#读取表格

In [1]:
import torch
import numpy as np
torch.set_printoptions(edgeitems=2, precision=2, linewidth=75)

In [2]:
# !pip install pandas

In [3]:
# import pandas as pd
# df = pd.read_csv("../data/p1ch4/tabular-wine/winequality-white.csv")

In [4]:
# df.head()

In [5]:
import csv
wine_path = "../data/p1ch4/tabular-wine/winequality-white.csv"


In [6]:
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=";",skiprows=1) # 分隔符为分号，跳过第一行，因为除了第一行，其他行都是数据

In [7]:
wineq_numpy

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]], dtype=float32)

In [8]:
col_list = next(csv.reader(open(wine_path), delimiter=";")) # 读取第一行，作为列名
wineq_numpy.shape,col_list

((4898, 12),
 ['fixed acidity',
  'volatile acidity',
  'citric acid',
  'residual sugar',
  'chlorides',
  'free sulfur dioxide',
  'total sulfur dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol',
  'quality'])

In [9]:
# torch到NumPy
wineq=torch.from_numpy(wineq_numpy)
wineq.shape,wineq.type()

(torch.Size([4898, 12]), 'torch.FloatTensor')

In [10]:
data=wineq[:,:-1] # <1> 前11列是特征
data,data.shape

(tensor([[ 7.00,  0.27,  ...,  0.45,  8.80],
         [ 6.30,  0.30,  ...,  0.49,  9.50],
         ...,
         [ 5.50,  0.29,  ...,  0.38, 12.80],
         [ 6.00,  0.21,  ...,  0.32, 11.80]]),
 torch.Size([4898, 11]))

In [11]:
target=wineq[:,-1] # <2> 最后一列是目标
target,target.shape

(tensor([6., 6.,  ..., 7., 6.]), torch.Size([4898]))

In [12]:
target=wineq[:,-1].long() # <3> 将浮点型转换为整型
target

tensor([6, 6,  ..., 7, 6])

In [13]:
target.min(),target.max() # 一共有7个等级

(tensor(3), tensor(9))

In [14]:
target_onehot=torch.zeros(target.shape[0],10) # <1> 创建一个全零张量

In [15]:
# target_onehot.scatter_(1,target.unsqueeze(1),1.0) # 

In [16]:
temp = target.unsqueeze(1)

In [17]:
target_onehot.scatter_(1,temp,1.0)

tensor([[0., 0.,  ..., 0., 0.],
        [0., 0.,  ..., 0., 0.],
        ...,
        [0., 0.,  ..., 0., 0.],
        [0., 0.,  ..., 0., 0.]])

In [18]:
target_onehot[0:3],target[0:3]

(tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]]),
 tensor([6, 6, 6]))

In [19]:
target_unsqueezed = target.unsqueeze(1)
target_unsqueezed

tensor([[6],
        [6],
        ...,
        [7],
        [6]])

In [20]:
data_mean=torch.mean(data,dim=0) # <1> 沿着第0维求平均值
data_mean
data_mean.shape # 全部压缩了

torch.Size([11])

In [21]:
data_var=torch.var(data,dim=0) # <2> 沿着第0维求方差
data_var

tensor([7.12e-01, 1.02e-02, 1.46e-02, 2.57e+01, 4.77e-04, 2.89e+02,
        1.81e+03, 8.95e-06, 2.28e-02, 1.30e-02, 1.51e+00])

In [22]:
data_normalized=(data-data_mean)/torch.sqrt(data_var) # <3> 标准化数据
data_normalized

tensor([[ 1.72e-01, -8.18e-02,  ..., -3.49e-01, -1.39e+00],
        [-6.57e-01,  2.16e-01,  ...,  1.34e-03, -8.24e-01],
        ...,
        [-1.61e+00,  1.17e-01,  ..., -9.63e-01,  1.86e+00],
        [-1.01e+00, -6.77e-01,  ..., -1.49e+00,  1.04e+00]])

In [23]:
bad_indexes = target <= 3 # <1> 找到所有的坏酒
bad_indexes,bad_indexes.shape,bad_indexes.sum()

(tensor([False, False,  ..., False, False]), torch.Size([4898]), tensor(20))

In [24]:
bad_data=data[bad_indexes] # <2> 从数据中取出坏酒
bad_data.shape

torch.Size([20, 11])

In [25]:
# 定义三种类型
bad_data = data[target <= 3]
mid_data = data[(target > 3) & (target < 7)] # <1>
good_data = data[target >= 7]

In [26]:
bad_mean = torch.mean(bad_data, dim=0)
mid_mean = torch.mean(mid_data, dim=0)
good_mean = torch.mean(good_data, dim=0)

In [27]:
bad_mean

tensor([7.60e+00, 3.33e-01, 3.36e-01, 6.39e+00, 5.43e-02, 5.33e+01,
        1.71e+02, 9.95e-01, 3.19e+00, 4.75e-01, 1.03e+01])

In [28]:
mid_mean

tensor([6.89e+00, 2.82e-01, 3.36e-01, 6.71e+00, 4.78e-02, 3.54e+01,
        1.42e+02, 9.94e-01, 3.18e+00, 4.87e-01, 1.03e+01])

In [29]:
good_mean

tensor([6.73e+00, 2.65e-01, 3.26e-01, 5.26e+00, 3.82e-02, 3.46e+01,
        1.25e+02, 9.92e-01, 3.22e+00, 5.00e-01, 1.14e+01])

In [30]:
# 使用 enumerate 和 zip 来同时迭代四个不同的列表
for i,args in enumerate(zip(col_list,bad_mean,mid_mean,good_mean)):
    print('{:2} {:20} {:6.2f} {:6.2f} {:6.2f}'.format(i,*args))

 0 fixed acidity          7.60   6.89   6.73
 1 volatile acidity       0.33   0.28   0.27
 2 citric acid            0.34   0.34   0.33
 3 residual sugar         6.39   6.71   5.26
 4 chlorides              0.05   0.05   0.04
 5 free sulfur dioxide   53.33  35.42  34.55
 6 total sulfur dioxide 170.60 141.83 125.25
 7 density                0.99   0.99   0.99
 8 pH                     3.19   3.18   3.22
 9 sulphates              0.47   0.49   0.50
10 alcohol               10.34  10.26  11.42


In [31]:
# fruits = ["apple", "banana", "cherry", "date"]

# for index, fruit in enumerate(fruits):
    # print(f"Index {index}: {fruit}")


In [32]:
total_sulfur_threshold = 141.83 # <1> 总硫量的阈值
total_sulfur_data = data[:,6] # <2> 取出总硫量这一列

In [33]:
predicted_indexes = torch.lt(total_sulfur_data, total_sulfur_threshold) # <3> 找到所有低于阈值的索引

In [34]:
predicted_indexes # 低于就是True，高于就是False

tensor([False,  True,  ...,  True,  True])

In [35]:
predicted_indexes.shape,predicted_indexes.sum() # 有多少个低于阈值的

(torch.Size([4898]), tensor(2727))

In [36]:
actual_indexes = target > 5 # 实际上是好酒的索引
actual_indexes.shape,actual_indexes.sum()

(torch.Size([4898]), tensor(3258))

In [38]:
n_matchs=torch.sum(actual_indexes & predicted_indexes).item() # <1> 找到预测正确的索引，转换为Python数值
n_matchs

2018

In [40]:
n_predicted=torch.sum(predicted_indexes).item() # <2> 找到所有预测为好酒的索引
n_predicted

2727

In [41]:
n_actual=torch.sum(actual_indexes).item() # <3> 找到所有实际上是好酒的索引
n_actual

3258

In [42]:
Precision=n_matchs/n_predicted # <1> 计算精度
Precision

0.74000733406674

In [44]:
Reccall=n_matchs/n_actual # <2> 计算召回率
Reccall

0.6193984039287906